"""
Script to compute zonal statistics for GEDI L4A points against a multiband HLS raster.
Steps:
1. Load GEDI L4A geospatial points (optionally buffer it into polygons - ## check errors on rastererization).
2. Load single band image - e.g. HLS composite raster for one band.
3. Reproject points to raster CRS and rasterize zones.
4. Compute per-zone statistics (e.g., mean) using xrspatial.zonal_stats.
5. Join results back to the input GeoDataFrame and export to GeoPackage.
"""

In [68]:
import os
import glob
import numpy as np
import geopandas as gpd
import rioxarray
from rasterio import features
from shapely.geometry import Point
from xrspatial import zonal_stats
import pandas as pd
from pyproj import CRS

In [86]:

def zonal_stats_raster(gpkg_points_path,img_path, ZONE_ID_COL = 'zone', BUFFER_METERS =0, band_name = 'b'):
        
    ####################################################
    # 1) Open raster as xarray (single band)
    ####################################################
    
    da = rioxarray.open_rasterio(img_path).squeeze()  # [y, x]
    if 'band' in da.dims:
        da = da.squeeze('band', drop=True)
        
    raster_crs = CRS.from_user_input(da.rio.crs)
    transform  = da.rio.transform()
    out_shape  = da.shape  # (rows, cols)
    nodata     = da.rio.nodata
    
    da = da.where(da != da.rio.nodata)
    
    
    
    ####################################################
    # 2) Read points
    ####################################################
    gdf = gpd.read_file(gpkg_points_path)
    
    # Create ID for the zonal stats
    if ZONE_ID_COL not in gdf.columns:
        gdf = gdf.reset_index(drop=True)
        gdf[ZONE_ID_COL] = np.arange(1, len(gdf) + 1, dtype=np.int32)
    
    
    
    ####################################################
    # 3) Reproject to raster CRS and buffer to polygons 
    ####################################################
    
    gdf_proj = gdf.to_crs(raster_crs)
    
    # BUffer
    if BUFFER_METERS > 0:
        gdf_proj["geometry"] = gdf_proj.geometry.buffer(BUFFER_METERS)
    
    
    ####################
    # 4) Build (geometry, id) tuples for rasterization
    ####################
    geom = list(zip(gdf_proj.geometry, gdf_proj[ZONE_ID_COL]))
    
    
    
    ####################################################
    # 5) Rasterize zones (same shape/transform as raster)
    ####################################################
    zones_arr = features.rasterize(
        geom,
        out_shape=out_shape,
        transform=transform,
        fill= nodata,
        nodata = nodata,
        masked = True,
        # fill=0,# background (no zone)
        dtype="int32",
        # all_touched=True
    )
    
    
    
    ####################################################
    # # 6) Wrap zones into an xarray aligned with the raster
    ####################################################
    
    zones_xarr = da.copy(deep=False)  # copies coords/attrs, not data
    zones_xarr.data = zones_arr
    # zones_xarr.attrs["nodata"] = nodata
    # zones_xarr
    
    
    ####################################################
    # 7) Compute zonal stats using xrspatial (min/max/mean/etc.)
    ####################################################
    
    # If your raster has nodata, pass it so stats ignore it
    zs_df = zonal_stats(
        zones=zones_xarr,      # your integer-labeled zone raster
        values=da,             # the raster with values to summarize
        # stats_funcs=['mean', 'max', 'min', 'sum', 'std', 'var', 'count'],
        stats_funcs=['mean'],
        nodata_values=nodata,   # very important → ensures nodata is ignored
        return_type='pandas.DataFrame'
    )
    
    
    ####################################################
    # # Keep only "mean" and rename the band
    ####################################################
    # zs_df = pd.DataFrame({"mean": pd.Series(zs["mean"])}).rename_axis(ZONE_ID_COL).reset_index()
    
    # Rename column -> e.g. "evi2_mean"
    zs_df = zs_df.rename(columns={"mean": f"{band_name}_mean"})
    
    
    ####################################################
    # 9) create output with zone id and values
    ####################################################
    # out = (gdf[[ZONE_ID_COL]]
    #        .drop_duplicates()
    #        .merge(zs_df, on=ZONE_ID_COL, how="left"))
    
   
    
    return zs_df
    
    

In [87]:
ref_year = 2022
ref_tile = 89

In [88]:
gpkg_points_path = f'/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/gedi/test/l4a_t90km_t{ref_tile}_veg{ref_year}_outrm.gpkg'

In [89]:
# Multiband image path
# img_path = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/yearly/br_af_grid90km_evi2_p95/tile_089/bands_vrt/HLS_89_01-01_12-30_2022_2022_percentile95.0evi2_Blue.vrt'

# Directory with your raster .vrt files
# dir_img = f'/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/yearly/br_af_grid90km_evi2_p95/tile_{ref_tile:03}/bands_vrt'
dir_img = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/yearly/br_af_grid90km_evi2_p95/tile_089/bands_vrt_HLS.89.percentile95.0evi2.2022/'
# dir_img

In [90]:
output_gpkg_zonalstats_fn = f'/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/gedi/test/l4a_t90km_t{ref_tile}_veg{ref_year}_outrm_zonal_HLS.gpkg'

In [91]:
# Define excluded band suffixes
exclude_bands = ['ValidMask.vrt', 'count.vrt', 'yearDate.vrt', 'JulianDate.vrt']


In [96]:
ZONE_ID_COL = 'zone'
BUFFER_METERS = 0

In [97]:
img_paths = glob.glob(os.path.join(dir_img, "*.vrt"))


img_paths = [
    f for f in img_paths
    if not any(f.endswith(ex) for ex in exclude_bands)
]

In [98]:
# img_paths[0]

In [99]:
# Load the base gdf
gdf = gpd.read_file(gpkg_points_path)



In [100]:
# Create ID for the zonal stats
if ZONE_ID_COL not in gdf.columns:
    gdf = gdf.reset_index(drop=True)
    gdf[ZONE_ID_COL] = np.arange(1, len(gdf) + 1, dtype=np.int32)

In [101]:
%%time
# Loop through rasters
for img_path in img_paths:
   
    # Get band name
    fname = os.path.basename(img_path)
    # -> "HLS_89_01-01_12-30_2022_2022_percentile95.0evi2_Blue.vrt"    
    fn_parts = fname.replace(".vrt", "").split("_")
    # ['HLS', '89', '01-01', '12-30', '2022', '2022', 'percentile95.0evi2', 'Blue']  
    # Rebuild the string to variable name
    band_name = f"{fn_parts[0]}.{fn_parts[4]}-{fn_parts[2]}.{fn_parts[5]}-{fn_parts[3]}.{fn_parts[-1]}"
    
    print(f'Zonal stats: {band_name}')

    
    # Run your function (it returns a gdf with ZONE_ID + stats)
    out = zonal_stats_raster(gpkg_points_path, img_path, ZONE_ID_COL, BUFFER_METERS, band_name)
    
    # print(f'Merge: {band_name}')
    # # Merge results back into main gdf (on zone_id)
    gdf = gdf.merge(out, on=ZONE_ID_COL, how="left")


# ~10k points 90 km tile
# Wall time: 2min 44s

Zonal stats: HLS.2022-01-01.2022-12-31.Blue
Zonal stats: HLS.2022-01-01.2022-12-31.EVI
Zonal stats: HLS.2022-01-01.2022-12-31.EVI2
Zonal stats: HLS.2022-01-01.2022-12-31.Green
Zonal stats: HLS.2022-01-01.2022-12-31.MSAVI
Zonal stats: HLS.2022-01-01.2022-12-31.NBR
Zonal stats: HLS.2022-01-01.2022-12-31.NBR2
Zonal stats: HLS.2022-01-01.2022-12-31.NDMI
Zonal stats: HLS.2022-01-01.2022-12-31.NDVI
Zonal stats: HLS.2022-01-01.2022-12-31.NIR
Zonal stats: HLS.2022-01-01.2022-12-31.Red
Zonal stats: HLS.2022-01-01.2022-12-31.SAVI
Zonal stats: HLS.2022-01-01.2022-12-31.SWIR
Zonal stats: HLS.2022-01-01.2022-12-31.SWIR2
Zonal stats: HLS.2022-01-01.2022-12-31.TCB
Zonal stats: HLS.2022-01-01.2022-12-31.TCG
Zonal stats: HLS.2022-01-01.2022-12-31.TCW
Zonal stats: HLS.2022-01-01.2022-12-31.Xgeo
Zonal stats: HLS.2022-01-01.2022-12-31.Ygeo
CPU times: user 1min 31s, sys: 16 s, total: 1min 47s
Wall time: 2min 59s


In [102]:
# gdf.columns

In [103]:
gdf_stats_nona = gdf.dropna()

In [104]:
len(gdf_stats_nona)

20105

In [105]:
# Export gdf_stats
gdf_stats_nona.to_file(output_gpkg_zonalstats_fn)


In [106]:
output_gpkg_zonalstats_fn

'/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/gedi/test/l4a_t90km_t89_veg2022_outrm_zonal_HLS.gpkg'

# Test

In [81]:
####################################################
# 1) Open raster as xarray (single band)
####################################################

da = rioxarray.open_rasterio(img_path).squeeze()  # [y, x]
if 'band' in da.dims:
    da = da.squeeze('band', drop=True)
    
raster_crs = CRS.from_user_input(da.rio.crs)
transform  = da.rio.transform()
out_shape  = da.shape  # (rows, cols)
nodata     = da.rio.nodata

da = da.where(da != da.rio.nodata)



####################################################
# 2) Read points
####################################################

# Create ID for the zonal stats
if ZONE_ID_COL not in gdf.columns:
    gdf = gdf.reset_index(drop=True)
    gdf[ZONE_ID_COL] = np.arange(1, len(gdf) + 1, dtype=np.int32)



####################################################
# 3) Reproject to raster CRS and buffer to polygons 
####################################################

gdf_proj = gdf.to_crs(raster_crs)

# BUffer
if BUFFER_METERS > 0:
    gdf_proj["geometry"] = gdf_proj.geometry.buffer(BUFFER_METERS)


####################
# 4) Build (geometry, id) tuples for rasterization
####################
geom = list(zip(gdf_proj.geometry, gdf_proj[ZONE_ID_COL]))



####################################################
# 5) Rasterize zones (same shape/transform as raster)
####################################################
zones_arr = features.rasterize(
    geom,
    out_shape=out_shape,
    transform=transform,
    fill= nodata,
    nodata = nodata,
    masked = True,
    # fill=0,# background (no zone)
    dtype="int32",
    # all_touched=True
)






In [82]:
####################################################
# # 6) Wrap zones into an xarray aligned with the raster
####################################################

zones_xarr = da.copy(deep=False)  # copies coords/attrs, not data
zones_xarr.data = zones_arr
# zones_xarr.attrs["nodata"] = nodata
# zones_xarr


####################################################
# 7) Compute zonal stats using xrspatial (min/max/mean/etc.)
####################################################

# If your raster has nodata, pass it so stats ignore it
zs_df = zonal_stats(
    zones=zones_xarr,      # your integer-labeled zone raster
    values=da,             # the raster with values to summarize
    # stats_funcs=['mean', 'max', 'min', 'sum', 'std', 'var', 'count'],
    stats_funcs=['mean'],
    nodata_values=nodata,   # very important → ensures nodata is ignored
    return_type='pandas.DataFrame'
)


In [83]:
zs_df.head()

,zone,mean
0,4.0,0.0103
1,48.0,0.0158
2,49.0,0.0146
3,50.0,0.0153
4,51.0,0.0145


In [84]:
gdf = gdf.merge(zs_df, on=ZONE_ID_COL, how="left")

In [85]:
gdf.columns

Index(['time', 'beam', 'elevation', 'shot_number', 'flags', 'sensitivity',
       'orbit', 'solar_elevation', 'track', 'agbd', 'year', 'month',
       'vegmask_2022', 'geometry', 'zone', 'mean'],
      dtype='object')

In [76]:




####################################################
# # Keep only "mean" and rename the band
####################################################
# zs_df = pd.DataFrame({"mean": pd.Series(zs["mean"])}).rename_axis(ZONE_ID_COL).reset_index()

# Rename column -> e.g. "evi2_mean"
zs_df = zs_df.rename(columns={"mean": f"{band_name}_mean"})


####################################################
# 9) create output with zone id and values
####################################################
out = (gdf[[ZONE_ID_COL]]
       .drop_duplicates()
       .merge(zs_df, on=ZONE_ID_COL, how="left"))



In [75]:
gdf.columns

Index(['time', 'beam', 'elevation', 'shot_number', 'flags', 'sensitivity',
       'orbit', 'solar_elevation', 'track', 'agbd', 'year', 'month',
       'vegmask_2020', 'geometry', 'zone'],
      dtype='object')

In [70]:
gdf.columns

Index(['time', 'beam', 'elevation', 'shot_number', 'flags', 'sensitivity',
       'orbit', 'solar_elevation', 'track', 'agbd', 'year', 'month',
       'vegmask_2020', 'geometry', 'zone'],
      dtype='object')